In [10]:
fimo_best="best_site.narrowPeak"
genbank_dir = "annotati_Refseq"
import os
import re
from Bio import SeqIO
from collections import Counter
from scipy.stats import hypergeom
import pandas as pd
diz={
    'C7Y66' : 'Chroococcidiopsis_sp._CCALA_051_(cyanobacteria)_GCF_003015105.gbff' ,
    'M5J74' : 'Chroococcidiopsis_sp._CCNUC1_(cyanobacteria)_GCF_023546805.gbff' ,
    'B1A85' : 'Chroococcidiopsis_sp._TS-821_(cyanobacteria)_GCF_002939305.gbff' ,
    'C7B79' : 'Chroococcidiopsis_cubana_CCALA_043_(cyanobacteria)_GCF_003003835.gbff' ,
    'CHRO' : 'Chroococcidiopsis_thermalis_PCC_7203_(cyanobacteria)_GCF_000317125.gbff' ,
    'CLLPBCKN' : 'Chroococcidiopsis_cubana_SAG_39.79_(cyanobacteria)_GCF_034438355.1.gbff' ,
    'LAU37' : 'Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff' ,
    'N4J56' : 'Chroococcidiopsis_sp._SAG_2025_(cyanobacteria)_GCF_032860985.gbff' ,
    'H6G17' : 'Chroococcidiopsis_sp._[FACHB-1243]_(cyanobacteria)_GCF_014696895.gbff' ,
}
with open(fimo_best,"r") as f:
    content=f.readlines()
names = [line.split()[0] for line in content]
GOs = []
GOs_background = []
go_to_genes = {}
i,j=0,0
for file2 in os.listdir(genbank_dir):
    genoma = SeqIO.parse(os.path.join(genbank_dir,file2), "genbank")
    for seq in genoma:
        for record in seq.features:
            if "gene" in record.qualifiers:
                gene=record.qualifiers["gene"][0]
                
                if record.qualifiers["locus_tag"][0] in names:
                    i+=1
                    go_terms = record.qualifiers.get("GO_process", ["None"])
                    #print(type(go_terms))
                    for go_term in go_terms:
                        if go_term != "None":
                            GOs.append(go_term)
                            # If the GO term is already in the dictionary, append the gene to its list
                            if go_term in go_to_genes:
                                go_to_genes[go_term].append(gene)
                            # If the GO term is not in the dictionary, add it with the gene as its first list item
                            else:
                                go_to_genes[go_term] = [gene]
                else:
                    j+=1
                    GOs_background.append([*record.qualifiers.get("GO_process", ["None"])])


GOs = [item for item in GOs if item != "None"]
GOs_background = [item for sublist in GOs_background for item in sublist if item != "None"]
GOs=Counter(GOs)
GOs_background=Counter(GOs_background)
#use hypergeometric test to calculate p-value
#M = total number of genes
#n = number of genes with the motif
#N = total number of genes with the GO term
#x = number of genes with the GO term
M = i+j
n = i
p_values = {}
enrichments = {}
for go in GOs:
    N = GOs_background[go]+GOs[go]
    x = GOs[go]
    enrichments[go] = (x/n)/(N/M)
    enrichments[go] = 'positive' if enrichments[go]>1 else 'negative'
    p_values[go] = hypergeom.sf(x-1, M, N, n)
    #uso distribuzione ipergeometrica per calcolare p-values
df = pd.DataFrame(list(p_values.items()), columns=['GO term', 'p-value'])
df['enrichment'] = df['GO term'].map(enrichments)
# Convert the lists of genes to strings for easier viewing in the DataFrame
for go_term, genes in go_to_genes.items():
    go_to_genes[go_term] = ', '.join(genes)

# Add a new column to the DataFrame with the names of genes associated with each GO term
df['Genes with Motif'] = df['GO term'].map(go_to_genes)
df = df.sort_values(by='p-value', ascending=True)
pd.set_option('display.max_rows', df.shape[0]+1)
print(f"Numero di geni con il motivo a monte: {n}")
print(f"Numero di geni senza il motivo a monte: {j}")
print(f"Numero di geni totale: {M}")
df

Numero di geni con il motivo a monte: 426
Numero di geni senza il motivo a monte: 14364
Numero di geni totale: 14790


,GO term,p-value,enrichment,Genes with Motif
12,GO:0006282 - regulation of DNA repair [Evidenc...,1.256269e-14,positive,"lexA, lexA, lexA, lexA, lexA, lexA, lexA, lexA..."
10,GO:0009432 - SOS response [Evidence IEA],1.256269e-14,positive,"recA, recA, recA, recA, recA, recA, recA, recA..."
25,GO:0007623 - circadian rhythm [Evidence IEA],5.366380e-10,positive,"kaiC, kaiC, kaiC, kaiC, kaiC, kaiC, kaiC"
26,GO:0046777 - protein autophosphorylation [Evid...,5.366380e-10,positive,"kaiC, kaiC, kaiC, kaiC, kaiC, kaiC, kaiC"
17,GO:0006793 - phosphorus metabolic process [Evi...,5.366380e-10,positive,"ppk2, ppk2, ppk2, ppk2, ppk2, ppk2, ppk2"
11,GO:0005978 - glycogen biosynthetic process [Ev...,4.304338e-08,positive,"glgB, glgB, glgB, glgB, glgB, glgB"
14,GO:0006265 - DNA topological change [Evidence ...,6.140755e-08,positive,"gyrB, gyrB, gyrB, gyrB, gyrB, topA, gyrB, topA..."
18,GO:0006260 - DNA replication [Evidence IEA],8.515331e-08,positive,"sbcC, sbcC, ligA, holA, priA, ligA, sbcC, ligA..."
19,GO:0008295 - spermidine biosynthetic process [...,3.786026e-07,positive,"speB, speB, speB, speB, speB, speB, speB"
9,GO:0006310 - DNA recombination [Evidence IEA],6.620199e-07,positive,"recA, sbcC, recA, sbcC, recA, recA, recA, sbcC..."
